In [138]:
import wrds 
import pandas as pd

In [139]:
conn = wrds.Connection() #connect to WRDS servers

Loading library list...
Done


In [140]:
conn.list_libraries().sort()
conn.list_tables(library='optionm') # open the optionmetrics database, theres 2 data sets we need, opprcd2020 and option_price_2020. 
                                    # the latter has the underlying price for the option trading on that day, so we need to merge them
                                    # based off of their optionid and the date, columns available in both datasets

['country',
 'currency',
 'distrd',
 'distribution',
 'distribution_projection',
 'distrprojd1996',
 'distrprojd1997',
 'distrprojd1998',
 'distrprojd1999',
 'distrprojd2000',
 'distrprojd2001',
 'distrprojd2002',
 'distrprojd2003',
 'distrprojd2004',
 'distrprojd2005',
 'distrprojd2006',
 'distrprojd2007',
 'distrprojd2008',
 'distrprojd2009',
 'distrprojd2010',
 'distrprojd2011',
 'distrprojd2012',
 'distrprojd2013',
 'distrprojd2014',
 'distrprojd2015',
 'distrprojd2016',
 'distrprojd2017',
 'distrprojd2018',
 'distrprojd2019',
 'distrprojd2020',
 'distrprojd2021',
 'exchange',
 'exchgd',
 'forward_price',
 'futures',
 'futures_price',
 'fwdprd1996',
 'fwdprd1997',
 'fwdprd1998',
 'fwdprd1999',
 'fwdprd2000',
 'fwdprd2001',
 'fwdprd2002',
 'fwdprd2003',
 'fwdprd2004',
 'fwdprd2005',
 'fwdprd2006',
 'fwdprd2007',
 'fwdprd2008',
 'fwdprd2009',
 'fwdprd2010',
 'fwdprd2011',
 'fwdprd2012',
 'fwdprd2013',
 'fwdprd2014',
 'fwdprd2015',
 'fwdprd2016',
 'fwdprd2017',
 'fwdprd2018',
 'fwdprd

In [141]:
optionsdata2020 = conn.raw_sql("""select symbol, strike_price, optionid, impl_volatility, open_interest, volume, 
                          vega, best_bid, best_offer, date, exdate, forward_price, secid
                          
                          from optionm.opprcd2020
                          
                          where volume > 500
                          and volume > 0.5 * open_interest
                          and cp_flag = 'C' 
                          and (exdate - date) <= 30 and (exdate-date) > 0 
                          and best_bid > 0.3
                          
                          LIMIT 5""")
                          

In [142]:
optionsdata2020

,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate,forward_price,secid
0,MATX 200717C40000,40000.0,134149787.0,1.183986,0.0,547.0,1.798065,0.90,1.35,2020-07-10,2020-07-17,None,101207.0
1,MATX 201120C60000,60000.0,136468370.0,0.517955,605.0,645.0,4.646450,1.30,1.70,2020-11-03,2020-11-20,None,101207.0
2,ARE 200117C160000,160000.0,128995057.0,NaN,1040.0,870.0,NaN,1.10,5.30,2020-01-16,2020-01-17,None,101209.0
3,ARE 200320C170000,170000.0,132076518.0,0.168499,101.0,522.0,16.129510,1.50,1.85,2020-02-24,2020-03-20,None,101209.0
4,ARE 200417C125000,125000.0,130122683.0,0.662608,3.0,600.0,10.762520,1.55,6.20,2020-03-23,2020-04-17,None,101209.0


In [143]:
specific = conn.raw_sql("""select *
                          
                          from optionm.secprd2020
                          where secid = 101138.0
                          LIMIT 10""")

In [144]:
specific

,secid,date,low,high,close,volume,return,cfadj,open,cfret,shrout
0,101138.0,2020-01-02,83.29,85.4700,83.99,551103.0,-0.008851,1.5,85.47,1.54095,49272.0
1,101138.0,2020-01-03,81.81,83.5600,82.95,551552.0,-0.012382,1.5,82.89,1.54095,49272.0
2,101138.0,2020-01-06,81.60,83.5600,83.33,376600.0,0.004581,1.5,82.26,1.54095,49272.0
3,101138.0,2020-01-07,82.04,83.7600,82.39,431987.0,-0.011280,1.5,83.43,1.54095,49272.0
4,101138.0,2020-01-08,82.00,83.4000,82.53,541683.0,0.001699,1.5,83.11,1.54095,49272.0
5,101138.0,2020-01-09,82.34,83.3800,82.93,670115.0,0.004847,1.5,83.15,1.54095,49272.0
6,101138.0,2020-01-10,82.02,84.3900,82.93,865823.0,0.000000,1.5,82.76,1.54095,49272.0
7,101138.0,2020-01-13,83.31,84.7700,84.55,399956.0,0.019535,1.5,83.49,1.54095,49272.0
8,101138.0,2020-01-14,84.04,85.3914,84.78,420873.0,0.002720,1.5,84.54,1.54095,49272.0
9,101138.0,2020-01-15,83.71,85.3700,84.69,456515.0,-0.001062,1.5,84.15,1.54095,49272.0


In [145]:
finaldata = conn.raw_sql("""select a.symbol, a.strike_price, a.optionid, a.impl_volatility, a.open_interest, a.volume, 
                          a.vega, a.best_bid, a.best_offer, a.date, a.exdate, b.close, b.volume
                          
                          
                          from optionm.opprcd2020 a
                          inner join optionm.secprd2020 b
                          
                          
                          on a.secid = b.secid
                          and a.date = b.date
                          
                          
                          where a.volume > 500
                          and a.volume > 0.5 * a.open_interest
                          and a.cp_flag = 'C' 
                          and (a.exdate - a.date) <= 30 and (a.exdate-a.date) > 0 
                          and a.best_bid > 0.15   
                          and (1 - ((b.close)/(a.strike_price/1000))) > .05 and (1 - ((b.close)/(a.strike_price/1000))) < .7
                          
                          order by a.date""") # join the 2 datasets together based on needed parameters

In [146]:
finaldata

,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate,close,volume
0,BABA 200110C232500,232500.0,131556054.0,0.266700,45.0,744.0,4.892366,0.31,0.32,2020-01-02,2020-01-10,219.77,15882903.0
1,AAPL 200110C327500,327500.0,131891483.0,0.316561,1837.0,1103.0,3.417485,0.17,0.21,2020-01-02,2020-01-10,300.35,33911864.0
2,AAPL 200110C325000,325000.0,131486336.0,0.307583,2969.0,2771.0,4.170783,0.23,0.26,2020-01-02,2020-01-10,300.35,33911864.0
3,VIX 200122C14000,14000.0,129717173.0,0.971107,5647.0,14738.0,1.268407,1.25,1.30,2020-01-02,2020-01-22,12.47,0.0
4,VIX 200122C14500,14500.0,131468829.0,1.023319,18520.0,19712.0,1.279927,1.10,1.15,2020-01-02,2020-01-22,12.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126865,IQ 210129C21000,21000.0,137398308.0,0.746413,55.0,10982.0,1.466653,0.41,0.44,2020-12-31,2021-01-29,17.48,7817321.0
126866,FUBO 210108C30000,30000.0,137735432.0,1.808542,52.0,8526.0,1.641329,2.18,2.20,2020-12-31,2021-01-08,28.00,63933896.0
126867,DIS 210108C200000,200000.0,137262238.0,0.385913,1150.0,1275.0,2.519423,0.18,0.19,2020-12-31,2021-01-08,181.18,7073495.0
126868,ICPT 210115C35000,35000.0,129851543.0,1.409880,725.0,2256.0,1.119168,0.40,0.50,2020-12-31,2021-01-15,24.70,1833122.0


In [147]:
df = pd.DataFrame(data=finaldata)
df.to_pickle("./optiondatafinal2020.pkl")
df.drop_duplicates(subset=['optionid'])


,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate,close,volume
0,BABA 200110C232500,232500.0,131556054.0,0.266700,45.0,744.0,4.892366,0.31,0.32,2020-01-02,2020-01-10,219.77,15882903.0
1,AAPL 200110C327500,327500.0,131891483.0,0.316561,1837.0,1103.0,3.417485,0.17,0.21,2020-01-02,2020-01-10,300.35,33911864.0
2,AAPL 200110C325000,325000.0,131486336.0,0.307583,2969.0,2771.0,4.170783,0.23,0.26,2020-01-02,2020-01-10,300.35,33911864.0
3,VIX 200122C14000,14000.0,129717173.0,0.971107,5647.0,14738.0,1.268407,1.25,1.30,2020-01-02,2020-01-22,12.47,0.0
4,VIX 200122C14500,14500.0,131468829.0,1.023319,18520.0,19712.0,1.279927,1.10,1.15,2020-01-02,2020-01-22,12.47,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126862,ENPH 210115C200000,200000.0,137198789.0,0.788166,720.0,7763.0,10.803800,3.45,3.50,2020-12-31,2021-01-15,175.47,5626524.0
126865,IQ 210129C21000,21000.0,137398308.0,0.746413,55.0,10982.0,1.466653,0.41,0.44,2020-12-31,2021-01-29,17.48,7817321.0
126866,FUBO 210108C30000,30000.0,137735432.0,1.808542,52.0,8526.0,1.641329,2.18,2.20,2020-12-31,2021-01-08,28.00,63933896.0
126868,ICPT 210115C35000,35000.0,129851543.0,1.409880,725.0,2256.0,1.119168,0.40,0.50,2020-12-31,2021-01-15,24.70,1833122.0


In [148]:
finaldata2 = conn.raw_sql("""select a.symbol, a.strike_price, a.optionid, a.impl_volatility, a.open_interest, a.volume, 
                          a.vega, a.best_bid, a.best_offer, a.date, a.exdate, b.close, b.volume, b.date
                          
                          
                          from optionm.opprcd2020 a
                          inner join optionm.secprd2020 b
                          
                          on a.secid = b.secid
                          and a.exdate = b.date
                          
                          
                          
                          where a.volume > 500
                          and a.optionid = 134149787.0
                          and a.volume > 0.5 * a.open_interest
                          and a.cp_flag = 'C' 
                          and (a.exdate - a.date) <= 30 and (a.exdate-a.date) > 0 
                          and a.best_bid > 0.15   
                          and (1 - ((b.close)/(a.strike_price/1000))) > .05
                          LIMIT 50""") # merge the 2 datasets together based on needed parameters

In [149]:
finaldata2

,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate,close,volume,date
0,MATX 200717C40000,40000.0,134149787.0,1.183986,0.0,547.0,1.798065,0.9,1.35,2020-07-10,2020-07-17,37.92,370702.0,2020-07-17


In [154]:
finaldata3 = conn.raw_sql("""select a.symbol, a.strike_price, a.optionid, a.impl_volatility, a.open_interest, a.volume, 
                          a.vega, a.best_bid, a.best_offer, a.date, a.exdate, b.close, b.volume, b.date
                          
                          
                          from optionm.opprcd2020 a
                          inner join optionm.secprd2020 b
                          
                          on a.secid = b.secid
                          and a.date = b.date
                          
                          
                          
                          where a.optionid = 131486336.0
                          """) # merge the 2 datasets together based on needed parameters

In [155]:
finaldata3

,symbol,strike_price,optionid,impl_volatility,open_interest,volume,vega,best_bid,best_offer,date,exdate,close,volume,date
0,AAPL 200110C325000,325000.0,131486336.0,0.307583,2969.0,2771.0,4.170783,0.23,0.26,2020-01-02,2020-01-10,300.35,33911864.0,2020-01-02
1,AAPL 200110C325000,325000.0,131486336.0,0.337601,3783.0,8941.0,2.882643,0.16,0.17,2020-01-03,2020-01-10,297.43,36633878.0,2020-01-03
2,AAPL 200110C325000,325000.0,131486336.0,0.357217,7364.0,4362.0,1.281290,0.06,0.07,2020-01-06,2020-01-10,299.80,29644644.0,2020-01-06
3,AAPL 200110C325000,325000.0,131486336.0,0.399388,10282.0,3471.0,0.701037,0.03,0.04,2020-01-07,2020-01-10,298.39,27877655.0,2020-01-07
4,AAPL 200110C325000,325000.0,131486336.0,0.391160,9929.0,5628.0,0.524054,0.02,0.03,2020-01-08,2020-01-10,303.19,33090946.0,2020-01-08
5,AAPL 200110C325000,325000.0,131486336.0,0.378536,9257.0,5152.0,0.335048,0.01,0.02,2020-01-09,2020-01-10,309.63,42621542.0,2020-01-09
6,AAPL 200110C325000,325000.0,131486336.0,NaN,7494.0,163.0,NaN,0.00,0.01,2020-01-10,2020-01-10,310.33,35217272.0,2020-01-10
